## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-02-19-57 +0000,nyt,"Trump Bows to Reality on Epstein Files Vote, i...",https://www.nytimes.com/2025/11/17/us/politics...
1,2025-11-18-02-09-00 +0000,wsj,The World’s Most-Watched YouTube Video Hasn’t ...,https://www.wsj.com/tech/the-worlds-most-watch...
2,2025-11-18-02-03-53 +0000,cbc,Canadian first baseman Josh Naylor agrees to 5...,https://www.cbc.ca/sports/baseball/mlb/canadia...
3,2025-11-18-02-03-30 +0000,bbc,Trump's plan for Gaza backed by UN Security Co...,https://www.bbc.com/news/articles/c0rpkgq4wllo...
4,2025-11-18-02-00-00 +0000,wsj,Trump’s grip on the GOP is showing signs of sl...,https://www.wsj.com/politics/policy/trumps-gri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,49
3,epstein,17
4,files,16
672,new,13
374,house,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
314,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,136
104,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...,127
324,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,120
275,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,114
202,2025-11-17-15-34-23 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
314,136,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
156,56,2025-11-17-18-40-35 +0000,nypost,Bangladesh’s ousted PM Sheikh Hasina sentenced...,https://nypost.com/2025/11/17/world-news/bangl...
103,45,2025-11-17-21-29-00 +0000,wsj,New York Officials to Team Up With Wall Street...,https://www.wsj.com/us-news/new-york-officials...
43,44,2025-11-17-23-51-34 +0000,wapo,U.N. Security Council approves Trump’s 20-poin...,https://www.washingtonpost.com/national-securi...
107,44,2025-11-17-21-22-00 +0000,wsj,Fed Vice Chair Philip Jefferson acknowledged t...,https://www.wsj.com/economy/central-banking/fe...
173,36,2025-11-17-17-39-35 +0000,nypost,Trump-supporting grocer says he may shut down ...,https://nypost.com/2025/11/17/business/trump-s...
127,36,2025-11-17-20-22-42 +0000,nypost,Ex-Disney star ripped for ‘demonic’ app that l...,https://nypost.com/2025/11/17/business/ex-disn...
24,34,2025-11-18-00-38-14 +0000,nypost,Wacky wigmaker Miriam Yarimi gets sweetheart p...,https://nypost.com/2025/11/17/us-news/wacky-wi...
119,30,2025-11-17-20-53-20 +0000,nypost,Trump opens door to military action on alleged...,https://nypost.com/2025/11/17/us-news/trump-op...
286,29,2025-11-17-09-32-23 +0000,bbc,Dozens of Indian pilgrims killed in bus accide...,https://www.bbc.com/news/articles/cvg7dme9en0o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
